In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm, skew

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, \
    RobustScaler, FunctionTransformer
from sklearn.linear_model import LinearRegression, LassoCV, Ridge, ElasticNetCV, ElasticNet
from sklearn.svm import SVR
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestRegressor, StackingRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error, make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split, cross_validate, cross_val_score, learning_curve
from sklearn.pipeline import Pipeline
from sklearn import set_config

In [2]:
np.random.seed(42)
set_config(display='diagram')

plt.rcParams['figure.figsize'] = (12, 8)
sns.set_theme(style='whitegrid')

In [3]:
train_df = trainDf = pd.read_csv('data/train.csv')
test_df = testDf = pd.read_csv('data/test.csv')

full_df = fullDf = pd.concat([trainDf, testDf], sort=True).reset_index(drop=True)

## Misc Functions

In [4]:
def plot_learning_curve(estimator, X_train, y_train, cv, train_sizes=np.linspace(0.1, 1, 10)):
    plt.style.use('seaborn-darkgrid')
    
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X_train, y_train, 
        scoring='neg_mean_squared_error',
        cv=cv, 
        n_jobs=-1, 
        train_sizes=train_sizes,
        shuffle=True,
        random_state=42
        )
    train_mean_scores = np.mean(train_scores, axis=1)
    test_mean_scores = np.mean(test_scores, axis=1)

    plt.title('Learning curve')
    plt.plot(train_sizes, train_mean_scores, 'y', label='Train Learning curve')
    plt.plot(train_sizes, test_mean_scores, 'b', label='Test Learning curve')
    plt.legend()

In [5]:
def neg_rmsle(y_true, y_pred):
    y_pred = np.abs(y_pred)
    
    return -1 * np.sqrt(mean_squared_log_error(y_true, y_pred))

In [6]:
def score_model(model, X, Y):
    
    scores = cross_validate(
        model, X, Y, 
        scoring=['r2', 'neg_mean_absolute_error', 'neg_mean_squared_error'], cv=2,
        n_jobs=-1, verbose=0)

    rmsle_score = cross_val_score(model, X, Y, cv=2, scoring=make_scorer(neg_rmsle))

    mse_score = -1 * scores['test_neg_mean_squared_error'].mean()
    mse_std = scores['test_neg_mean_squared_error'].std()

    mae_score = -1 * scores['test_neg_mean_absolute_error'].mean()
    mae_std = scores['test_neg_mean_absolute_error'].std()

    r2_score_mean = scores['test_r2'].mean()
    r2_std = scores['test_r2'].std()

    print('[CV] MSE: %.4f (%.4f)' % (mse_score, mse_std))
    print('[CV] MAE: %.4f (%.4f)' % (mae_score, mae_std))
    print('[CV] R^2: %.4f (%.4f)' % (r2_score_mean, r2_std))
    print('[CV] RMSLE: %.6f (%.4f)' % (-1 * rmsle_score.mean(), rmsle_score.std()))

In [7]:
def get_columns_from_transformer(column_transformer, input_colums):    
    col_name = []

    for transformer_in_columns in column_transformer.transformers_[:-1]: #the last transformer is ColumnTransformer's 'remainder'
        raw_col_name = transformer_in_columns[2]
        if isinstance(transformer_in_columns[1],Pipeline): 
            transformer = transformer_in_columns[1].steps[-1][1]
        else:
            transformer = transformer_in_columns[1]
        try:
            names = transformer.get_feature_names(raw_col_name)
        except AttributeError: # if no 'get_feature_names' function, use raw column name
            names = raw_col_name
        if isinstance(names,np.ndarray): # eg.
            col_name += names.tolist()
        elif isinstance(names,list):
            col_name += names    
        elif isinstance(names,str):
            col_name.append(names)

    [_, _, reminder_columns] = column_transformer.transformers_[-1]

    for col_idx in reminder_columns:
        col_name.append(input_colums[col_idx])

    return col_name

## Data Cleaning and Preparation

In [8]:
num_features = [f for f in full_df.columns if full_df.dtypes[f] != 'object']
num_features.remove('Id')
num_features.remove('SalePrice')

cat_features = [f for f in full_df.columns if full_df.dtypes[f] == 'object']

In [9]:
for feature in (
    'PoolQC', 
    'FireplaceQu', 
    'Alley', 
    'Fence', 
    'MiscFeature', 
    'BsmtQual', 
    'BsmtCond', 
    'BsmtExposure', 
    'BsmtFinType1', 
    'BsmtFinType2',
    'GarageType', 
    'GarageFinish', 
    'GarageQual', 
    'GarageCond',
    'BsmtQual', 
    'BsmtCond', 
    'BsmtExposure', 
    'BsmtFinType1', 
    'BsmtFinType2',
    'MasVnrType',
    'MSSubClass',
):
    train_df[feature] = train_df[feature].fillna('None')
    test_df[feature] = test_df[feature].fillna('None')
    full_df[feature] = full_df[feature].fillna('None')

for feature in (
    'BsmtFinSF1', 
    'BsmtFinSF2', 
    'BsmtUnfSF',
    'TotalBsmtSF', 
    'BsmtFullBath', 
    'BsmtHalfBath',
    'MasVnrArea',
    'GarageCars',
    'GarageArea',
    'GarageYrBlt',
):
    train_df[feature] = train_df[feature].fillna(0)
    test_df[feature] = test_df[feature].fillna(0)
    full_df[feature] = full_df[feature].fillna(0)

for feature in (
    'Electrical', 
    'KitchenQual', 
    'Exterior1st',
    'Exterior2nd', 
    'SaleType',
    'MSZoning',
    'Utilities',
):
    train_df[feature] = train_df[feature].fillna(train_df[feature].mode()[0])
    test_df[feature] = test_df[feature].fillna(test_df[feature].mode()[0])
    full_df[feature] = full_df[feature].fillna(test_df[feature].mode()[0])

train_df['Functional'] = train_df['Functional'].fillna('Typ')
test_df['Functional'] = test_df['Functional'].fillna('Typ')
full_df['Functional'] = full_df['Functional'].fillna('Typ')

In [10]:
for dataframe in [train_df, test_df]:
    dataframe['Exterior1st'].replace(['Brk Cmn', 'CmentBd', 'Wd Shng'], ['BrkComm', 'CemntBd', 'Wd Sdng'], inplace=True)
    dataframe['Exterior2nd'].replace(['Brk Cmn', 'CmentBd', 'Wd Shng'], ['BrkComm', 'CemntBd', 'Wd Sdng'], inplace=True)

In [11]:
ordinal_feature_mapping = {
    'ExterQual': {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}, 
    'ExterCond': {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4},
    'BsmtQual': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'BsmtCond': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'BsmtFinType1': {'None': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6},
    'BsmtFinType2': {'None': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6},
    'HeatingQC': {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4},
    'KitchenQual': {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4},
    'FireplaceQu': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'GarageFinish': {'None': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3},
    'GarageQual': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'GarageCond': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'PoolQC': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'Fence': {'None': 0, 'MnWw': 1, 'GdWo': 2, 'MnPrv': 3, 'GdPrv': 4},
    'PavedDrive': {'N': 0, 'P': 1, 'Y': 2},
    'CentralAir': {'N': 0, 'Y': 1},
    'Alley': {'None': 0, 'Pave': 1, 'Grvl': 2},
    'Street': {'Pave': 0, 'Grvl': 1},
    'Functional': {'Sal': 0, 'Sev': 1, 'Maj2': 2, 'Maj1': 3, 'Mod': 4, 'Min2': 5, 'Min1': 6, 'Typ': 7}
}

non_ordinal_cat_features = list(set(cat_features) - set(ordinal_feature_mapping.keys()))

for cat_feature in non_ordinal_cat_features:
    train_df[cat_feature + 'Enc'] = LabelEncoder().fit_transform(train_df[cat_feature])
    test_df[cat_feature + 'Enc'] = LabelEncoder().fit_transform(test_df[cat_feature])

for ordinal_feature, feature_mapping in ordinal_feature_mapping.items():
    train_df[ordinal_feature + 'Enc'] = train_df[ordinal_feature].map(feature_mapping)
    test_df[ordinal_feature + 'Enc'] = test_df[ordinal_feature].map(feature_mapping)

### Num Feature Scaling

In [12]:
# https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard

## Feature Engineering

In [13]:
for dataframe in [train_df, test_df]:
    dataframe['Has2ndFloor'] = dataframe['2ndFlrSF'].apply(lambda x: int(x > 0))
    dataframe['HasBsmnt'] = dataframe['TotalBsmtSF'].apply(lambda x: int(x > 0))
    dataframe['HasGarage'] = dataframe['GarageArea'].apply(lambda x: int(x > 0))
    dataframe['HasPool'] = dataframe['PoolArea'].apply(lambda x: int(x > 0))
    dataframe['HasFence'] = dataframe['Fence'].apply(lambda x: int(x != 'None'))
    dataframe['HasFireplace'] = dataframe['Fireplaces'].apply(lambda x: int(x > 0))
    dataframe['HasMasVnr'] = dataframe['MasVnrType'].apply(lambda x: int(x != 'None'))
    dataframe['HasShed'] = (dataframe['MiscFeature'] == 'Shed') * 1

    dataframe['HouseAge'] = dataframe['YrSold'].astype('int') - dataframe['YearBuilt'].astype('int')
    dataframe['HouseAgeSinRemod'] = dataframe['YrSold'].astype('int') - dataframe['YearRemodAdd'].astype('int')

In [14]:
# Remove outliers
train_df.drop(
    train_df[(train_df['GrLivArea'] > 4000) & (train_df['SalePrice'] < 700000)].index
);

In [15]:
subclassCategories = [20, 30, 40, 45, 50, 60, 70, 75, 80, 85, 90, 120, 160, 180, 190] #removed 150 class as useless during predictions
basementFinishCategories = ['None', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ']
electricalCategories = ['Mix', 'FuseP', 'FuseF', 'FuseA', 'SBrkr']

exteriorCategories = ['AsbShng', 'AsphShn', 'BrkComm', 'BrkFace', 'CBlock', 'CemntBd', 'HdBoard', 'ImStucc', 'MetalSd', 'Other', 'Plywood', 'PreCast', 'Stone', 'Stucco', 'VinylSd', 'Wd Sdng', 'WdShing']

conditionCategories = train_df['Condition1'].unique()
neighborhoodCategories = full_df['Neighborhood'].unique()
saleCondCategories = full_df['SaleCondition'].unique()
garageTypeCategories = full_df['GarageType'].unique()
lotConfigCategories = full_df['LotConfig'].unique() # feature was removed from the model
lotShapeCategories = full_df['LotShape'].unique()
landSlopeCategories = full_df['LandSlope'].unique()

In [16]:
year_built_bins = np.linspace(1871, 2010, 10) # 10 bins

In [17]:
# Reference: https://www.kaggle.com/cerberus4229/voting-regressor-with-pipelines

for dataframe in [train_df, test_df]:
    dataframe['TotalBathrooms'] = (dataframe['FullBath'] + (0.5 * dataframe['HalfBath']) +
                                dataframe['BsmtFullBath'] + (0.5 * dataframe['BsmtHalfBath']))

    dataframe['OverallHouseQCBin'] = pd.qcut(dataframe['OverallQual'] + dataframe['OverallCond'], q=3, labels=[0, 1, 2]) # performs worse than OverallQual
    dataframe['IsPavedDrive'] = (dataframe['PavedDrive'] == 'Y') * 1 # performs worse
    dataframe['IsNeighborhoodElite'] = (dataframe['Neighborhood'].isin(['NridgHt', 'CollgeCr', 'Crawfor', 'StoreBr', 'Timber'])) * 1 # worse
    dataframe['YearBuiltBin'] = pd.cut(dataframe['YearBuilt'], bins=year_built_bins, labels=range(1, 10)) 
    dataframe['KitchenQCBin'] = pd.cut(dataframe['KitchenQualEnc'] * dataframe['KitchenAbvGr'], [-1, 2, 7], labels=[0, 1])

    dataframe['IsFunctional'] = (dataframe['Functional'] == 'Typ') * 1
    dataframe['FunctionalGroup'] = dataframe['Functional'].map({
        'Typ': 2,
        'Min1': 1,
        'Min2': 1,
        'Mod': 1,
        'Maj1': 0,
        'Maj2': 0,
        'Sev': 0,
        'Sav': 0,
    })

    dataframe['IsHeatingGood'] = dataframe['HeatingQC'].map({'Po': 0, 'Fa': 0, 'TA': 1, 'Gd': 1, 'Ex': 1})
    dataframe['IsModernHouseType'] = dataframe['MSSubClass'].map({20: 1, 30: 0, 40: 0, 45: 0,50: 0, 60: 1, 70: 0, 75: 0, 80: 0, 85: 0,
         90: 0, 120: 1, 150: 0, 160: 0, 180: 0, 190: 0})
    dataframe['IsLandFlat'] = (dataframe['LandContour'] == 'Lvl') * 1
    dataframe['IsNewHouseSold'] = (full_df['YearBuilt'] == full_df['YrSold'].astype(int)) * 1
    dataframe['External'] = (full_df['YearBuilt'] == full_df['YrSold'].astype(int)) * 1

    dataframe['IsExterCondGood'] = dataframe['ExterCond'].map({'Po': 0, 'Fa': 0, 'TA': 1, 'Gd': 1, 'Ex': 1})

In [18]:
# https://www.kaggle.com/humananalog/xgboost-lasso

In [19]:
baseline_features = [
    '1stFlrSF',
    '2ndFlrSF',
    'BsmtFinSF1', 
    'BsmtFinSF2',
    'BsmtUnfSF', 
    'BsmtFinType1Enc',
    'BsmtFinType2Enc',
    'OverallQual',
    'GarageCars',
    'OverallCond', 
    'Neighborhood',
    'MSSubClass', 
    'LotShape',
    'LandSlope',
    'BsmtCondEnc',
    'BsmtQualEnc', 
    'SaleCondition',
    'CentralAirEnc',
    'Condition1',
    'Condition2',
    'TotalBathrooms',
    'GarageFinishEnc',
    'KitchenQualEnc',
    'BedroomAbvGr',
    'MSZoning',
    'ExterQualEnc',
    'IsNewHouseSold',
    'LandContour',
    'HasFireplace',
    'FunctionalGroup',
    'HouseAge',
    'FenceEnc',
    'IsModernHouseType',
]

X = train_df[baseline_features]
Y = train_df['SalePrice']

## Modeling

In [20]:
# Build feature transformer

logTransformer = FunctionTransformer(func=np.log1p, inverse_func=np.expm1)

featureTransformer = ColumnTransformer([
        ('basement_area_log', logTransformer, ['BsmtFinSF1', 'BsmtFinSF2']),
        ('neighborhood_onehot', OneHotEncoder(categories=[neighborhoodCategories]), ['Neighborhood']),
        ('subclass_onehot', OneHotEncoder(categories=[subclassCategories], handle_unknown='ignore'), ['MSSubClass']),
        ('lot_shape_onehot', OneHotEncoder(categories=[lotShapeCategories]), ['LotShape']),
        ('land_slope_onehot', OneHotEncoder(categories=[landSlopeCategories]), ['LandSlope']),
        ('sale_condtion_onehot', OneHotEncoder(categories=[saleCondCategories]), ['SaleCondition']),
        ('land_contour_onehot', OneHotEncoder(), ['LandContour']),
        ('zoning_onehot', OneHotEncoder(), ['MSZoning']),
        ('condition_onehot', OneHotEncoder(categories=[conditionCategories, conditionCategories]), ['Condition1', 'Condition2']), #
    ],
    remainder='passthrough'
)

### ElasticNet

In [21]:
elasticnet_model = ElasticNet(
    alpha=0.5,
    l1_ratio=0.8,
    random_state=42,
)

elasticnet_pipeline = Pipeline([
    ('preprocessing', featureTransformer),
    ('elasticnet', elasticnet_model)
])

score_model(elasticnet_pipeline, X, Y)

[CV] MSE: 1157170137.3388 (87064118.3079)
[CV] MAE: 20166.3290 (609.2098)
[CV] R^2: 0.8148 (0.0254)
[CV] RMSLE: 0.205980 (0.0426)


In [22]:
parameters = {
    'elasticnet__alpha': [9e-4, 7e-4, 5e-4, 3e-4, 1e-4, 1e-3, 5e-2, 1e-2, 0.1, 0.5, 1, 2, 5],
    'elasticnet__l1_ratio': [0.01, 0.1, 0.2, 0.3, 0.5, 0.6, 0.7, 0.8, 1],
}

paramSearch = GridSearchCV(
  estimator=elasticnet_pipeline,
  scoring='neg_mean_squared_error',
  param_grid=parameters, 
  cv=2,
  n_jobs=-1, 
  verbose=3
)

#paramSearch.fit(X, Y)
#paramSearch.best_params_, paramSearch.best_score_

### Ridge

In [23]:
# https://www.kaggle.com/marktsvirko/votingregressor-xgb-svm-top-10

In [24]:
ridge_pipeline = Pipeline([
    ('preprocessing', featureTransformer),
    ('ridge', Ridge(
        solver='auto',
        alpha=522,
        random_state=42,
    )),
])

score_model(ridge_pipeline, X, Y)

[CV] MSE: 1367540151.6855 (87592301.5338)
[CV] MAE: 22127.9798 (438.3611)
[CV] R^2: 0.7813 (0.0275)
[CV] RMSLE: 0.177489 (0.0007)


In [25]:
parameters = {
    'ridge__solver': ['auto', 'saga', 'sag', 'cholesky'],
    'ridge__alpha': [510, 519, 520, 522, 530],
}

paramSearch = GridSearchCV(
  estimator=ridge_pipeline,
  scoring=make_scorer(neg_rmsle),
  param_grid=parameters, 
  cv=2,
  n_jobs=-1, 
  verbose=3
)

#paramSearch.fit(X, Y)
#paramSearch.best_params_, paramSearch.best_score_

### SVR

In [65]:
svr_model = SVR(
    kernel='linear',
    C=200
)

svr_pipeline = Pipeline([
    ('preprocessing', featureTransformer),
    ('svr', svr_model),
])

score_model(svr_pipeline, X, Y)

[CV] MSE: 1332462671.5171 (20680320.6937)
[CV] MAE: 20559.8665 (866.9183)
[CV] R^2: 0.7875 (0.0165)
[CV] RMSLE: 0.163320 (0.0088)


In [64]:
parameters = {
    'svr__C': [1, 20, 50, 100, 120, 150, 200],
    'svr__kernel': ['linear'],
    #'svr__tol': [1e-3, 1e-5, 1e-4, 1e-2, 0.1]
}

paramSearch = GridSearchCV(
   estimator=svr_pipeline,
   scoring=make_scorer(neg_rmsle),
   param_grid=parameters,
   cv=2,
   n_jobs=-1, 
   verbose=3
)

#paramSearch.fit(X, Y)
#paramSearch.best_params_, paramSearch.best_score_

Fitting 2 folds for each of 7 candidates, totalling 14 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  14 | elapsed:    1.3s remaining:    4.6s
[Parallel(n_jobs=-1)]: Done   8 out of  14 | elapsed:    4.9s remaining:    3.7s
[Parallel(n_jobs=-1)]: Done  14 out of  14 | elapsed:   19.9s finished


({'svr__C': 200, 'svr__kernel': 'linear'}, -0.16332046865053396)

### Random Forest

In [26]:
random_forest_model = RandomForestRegressor(
        bootstrap=True,
        max_depth=15,
        max_features='sqrt',
        max_leaf_nodes=None,
        max_samples=None,
        min_samples_leaf=5,
        min_samples_split=5,
        n_estimators=2500,
        random_state=42,
    )

random_forest_pipeline = Pipeline([
    ('preprocessing', featureTransformer),
    ('random_forest', random_forest_model),
])

# Measure performance

score_model(random_forest_pipeline, X, Y)

## RF(max_depth=15, max_features='sqrt', min_samples_leaf=1, min_samples_split=3, n_estimators=3650)
## CV=2, ?
#[CV] MSE: 904073278.0912 (34098840.2146)
#[CV] MAE: 17896.4251 (795.0885)
#[CV] R^2: 0.8563 (0.0036)

## RF(max_depth=25, max_features='sqrt', min_samples_leaf=1, min_samples_split=2, n_estimators=1000)
## CV=2, 0.15007
#[CV] MSE: 889831012.8445 (46061331.0130)
#[CV] MAE: 17721.0043 (817.3113)
#[CV] R^2: 0.8587 (0.0015)

#[CV] MSE: 901503923.2650 (37758910.2379)
#[CV] MAE: 17843.5568 (877.0261)
#[CV] R^2: 0.8568 (0.0029)
#[CV] RMSLE: 0.148084 (0.0074)

[CV] MSE: 1209359844.2069 (1843381.5482)
[CV] MAE: 20454.0732 (702.2607)
[CV] R^2: 0.8073 (0.0123)
[CV] RMSLE: 0.167425 (0.0069)


In [27]:
random_forest_pipeline.fit(X, Y)

X_columns = get_columns_from_transformer(random_forest_pipeline.named_steps['preprocessing'], list(X.columns))

features_list = sorted(zip(random_forest_pipeline.named_steps['random_forest'].feature_importances_, X_columns), reverse=True)
random_forest_pipeline.named_steps['preprocessing'].transformers_

features_list

[(0.15379866259663588, 'OverallQual'),
 (0.08924538182613381, 'GarageCars'),
 (0.08834210226004628, '1stFlrSF'),
 (0.08218667685263008, 'ExterQualEnc'),
 (0.07404793453297402, 'KitchenQualEnc'),
 (0.06704699848018082, 'BsmtQualEnc'),
 (0.06191199969956158, 'TotalBathrooms'),
 (0.0601347752697437, 'HouseAge'),
 (0.047125272331239344, 'BsmtFinSF1'),
 (0.04334885802901738, '2ndFlrSF'),
 (0.03477371623261905, 'GarageFinishEnc'),
 (0.030483798125175966, 'HasFireplace'),
 (0.018980817090073248, 'BsmtFinType1Enc'),
 (0.015534237901662354, 'MSSubClass_60'),
 (0.014113914120704276, 'BsmtUnfSF'),
 (0.013178184922223286, 'Neighborhood_NridgHt'),
 (0.011945571476499692, 'BedroomAbvGr'),
 (0.011202688424742775, 'Neighborhood_NoRidge'),
 (0.010234622969908548, 'IsModernHouseType'),
 (0.007208882132253255, 'SaleCondition_Partial'),
 (0.0063406995841250825, 'OverallCond'),
 (0.005509646453560206, 'MSZoning_RM'),
 (0.0051346767719294304, 'LotShape_Reg'),
 (0.004620836780751636, 'MSZoning_RL'),
 (0.0038

In [28]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_objective, plot_histogram

parameters = {
    'random_forest__max_depth': [None, 15,  20, 25],
    'random_forest__max_features': ['sqrt'],
    'random_forest__max_leaf_nodes': [None, 5, 15, 17, 20],
    'random_forest__min_samples_leaf': [1, 2, 3],
    'random_forest__min_samples_split': [2, 1, 3],
    'random_forest__max_samples': [None, 2, 5],
    'random_forest__n_estimators': [1500, 2000, 2500]
}

paramSearch = GridSearchCV(
  estimator=random_forest_pipeline,
  scoring=make_scorer(neg_rmsle),
  param_grid=parameters, 
  cv=2,
  n_jobs=-1, 
  verbose=3
)

#paramSearch.fit(X, Y)
#paramSearch.best_params_, paramSearch.best_score_

In [29]:
#plot_learning_curve(random_forest_pipeline, X, Y, cv=3)

### XGBoostRegressor

In [30]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(
        max_depth=5,
        n_estimators=7050,
        learning_rate=0.01,
        min_child_weight=1.5,
        subsample=0.2,
        gamma=0,
        reg_alpha=1,
        reg_lambda=0.1,
        objective='reg:gamma',
        booster='gbtree'
    )

xgb_pipeline = Pipeline([
    ('preprocessing', featureTransformer),
    ('xgb_regressor', xgb_model),
])

# Measure Performance

score_model(xgb_pipeline, X, Y)

[CV] MSE: 732641130.9703 (5344883.1889)
[CV] MAE: 15646.3619 (283.7011)
[CV] R^2: 0.8832 (0.0081)
[CV] RMSLE: 0.125072 (0.0047)


In [31]:
xgb_pipeline.fit(X, Y)
X_columns = get_columns_from_transformer(xgb_pipeline.named_steps['preprocessing'], list(X.columns))

features_list = sorted(zip(xgb_pipeline.named_steps['xgb_regressor'].feature_importances_, X_columns), reverse=True)
features_list

[(0.10488179, 'OverallQual'),
 (0.06410893, 'ExterQualEnc'),
 (0.057148222, 'GarageCars'),
 (0.055850655, 'TotalBathrooms'),
 (0.040229782, 'KitchenQualEnc'),
 (0.039311953, 'HasFireplace'),
 (0.03448331, 'MSZoning_C (all)'),
 (0.026530785, 'CentralAirEnc'),
 (0.023906965, 'BsmtQualEnc'),
 (0.020864632, '1stFlrSF'),
 (0.018612808, 'MSSubClass_30'),
 (0.017896276, 'BsmtFinSF1'),
 (0.013970452, 'SaleCondition_Family'),
 (0.013664116, 'MSSubClass_180'),
 (0.013484347, '2ndFlrSF'),
 (0.01347552, 'GarageFinishEnc'),
 (0.013446186, 'LotShape_IR3'),
 (0.013141641, 'Neighborhood_IDOTRR'),
 (0.011748367, 'MSZoning_RM'),
 (0.010780746, 'MSSubClass_45'),
 (0.0107093705, 'SaleCondition_Abnorml'),
 (0.009700691, 'MSSubClass_60'),
 (0.009420548, 'Neighborhood_OldTown'),
 (0.009098041, 'Condition1_PosN'),
 (0.009053719, 'Condition2_Norm'),
 (0.008716666, 'SaleCondition_Alloca'),
 (0.008633941, 'Condition2_Feedr'),
 (0.008584648, 'OverallCond'),
 (0.008550188, 'Neighborhood_MeadowV'),
 (0.00788855, 'N

In [32]:
parameters = {
    'xgb_regressor__objective': ['reg:gamma'], # 'reg:squarederror', 'reg:squaredlogerror'
    'xgb_regressor__learning_rate': [0.01],
    'xgb_regressor__n_estimators': [6900, 7000, 7100],
    'xgb_regressor__max_depth': [5],
    'xgb_regressor__booster': ['gbtree'],
    'xgb_regressor__min_child_weight': [1.5],
    'xgb_regressor__gamma': [0],
    'xgb_regressor__subsample': [0.2],
    'xgb_regressor__reg_alpha': [1],
    'xgb_regressor__reg_lambda': [0.1],
}

paramSearch = GridSearchCV(
   estimator=xgb_pipeline,
   scoring=make_scorer(neg_rmsle),
   param_grid=parameters,
   cv=2,
   n_jobs=-1, 
   verbose=3
)

#paramSearch.fit(X, Y)
#paramSearch.best_params_, paramSearch.best_score_

#({'xgb_regressor__booster': 'gbtree',
#  'xgb_regressor__gamma': 0.03,
#  'xgb_regressor__learning_rate': 0.04,
#  'xgb_regressor__max_depth': 5,
#  'xgb_regressor__min_child_weight': 2,
#  'xgb_regressor__n_estimators': 2000,
#  'xgb_regressor__objective': 'reg:gamma',
#  'xgb_regressor__subsample': 0.7},
# -0.1293080016874709)

#({'xgb_regressor__booster': 'gbtree',
#  'xgb_regressor__gamma': 0.03,
#  'xgb_regressor__learning_rate': 0.06,
#  'xgb_regressor__max_depth': 5,
#  'xgb_regressor__min_child_weight': 2,
#  'xgb_regressor__n_estimators': 2000,
#  'xgb_regressor__objective': 'reg:gamma',
#  'xgb_regressor__subsample': 0.7},
# -0.13023288188100762)

#({'xgb_regressor__booster': 'gbtree',
#  'xgb_regressor__gamma': 0.02,
#  'xgb_regressor__learning_rate': 0.07,
#  'xgb_regressor__max_depth': 2,
#  'xgb_regressor__min_child_weight': 1,
#  'xgb_regressor__n_estimators': 800,
#  'xgb_regressor__objective': 'reg:gamma',
#  'xgb_regressor__subsample': 0.8},
# -0.12779856264607697)

### StackRegressor

In [33]:
# https://www.kaggle.com/lavanyashukla01/how-i-made-top-0-3-on-a-kaggle-competition

In [34]:
stacking_final_model = XGBRegressor()

stacking_model = StackingRegressor(
    estimators=[
        ('xgb', xgb_model),
        ('random_forest', random_forest_model),
        ('elasticnet', elasticnet_model),
    ], 
    #final_estimator=stacking_final_model,
    cv=2,
    n_jobs=-1,
)

stacking_regressor_pipeline = Pipeline([
    ('preprocessing', featureTransformer),
    ('stacking_regressor', stacking_model)
])

score_model(stacking_regressor_pipeline, X, Y)

[CV] MSE: 783809380.8984 (49247690.4325)
[CV] MAE: 16202.6767 (55.4011)
[CV] R^2: 0.8746 (0.0156)
[CV] RMSLE: 0.131372 (0.0000)


### VotingRegressor

In [41]:
# https://www.kaggle.com/marktsvirko/votingregressor-xgb-svm-top-10

voting_model = VotingRegressor(
    estimators=[
        ('xgb', xgb_model), 
        ('random_forest', random_forest_model),
        ('elasticnet', elasticnet_model),
    ],
    n_jobs=-1,
)

voting_pipeline = Pipeline([
    ('preprocessing', featureTransformer),
    ('voting_regressor', voting_model)
])

score_model(voting_pipeline, X, Y)

[CV] MSE: 867687198.2568 (1833083.8132)
[CV] MAE: 16684.2830 (645.2620)
[CV] R^2: 0.8618 (0.0089)
[CV] RMSLE: 0.137184 (0.0065)


In [45]:
parameters = {
    'voting_regressor__weights': [None, [0.6, 0.3, 0.1]],
}

paramSearch = GridSearchCV(
   estimator=voting_pipeline,
   scoring=make_scorer(neg_rmsle),
   param_grid=parameters,
   cv=2,
   n_jobs=-1, 
   verbose=3
)

#paramSearch.fit(X, Y)
#paramSearch.best_params_, paramSearch.best_score_

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  16 | elapsed:    0.2s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done   9 out of  16 | elapsed:    0.3s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:   11.8s finished


({'voting_regressor__weights': None}, -0.13718441242581908)

## Predict Submissions

In [36]:
random_forest_pipeline.fit(X, Y)

x_test = test_df[baseline_features]
y_test_predicted = random_forest_pipeline.predict(x_test)

y_test_predicted = np.rint(y_test_predicted).astype(int)

submission_df = pd.DataFrame({
    'Id': test_df['Id'],
    'SalePrice': y_test_predicted,
})

submission_df.to_csv('./data/submission_random_forest.csv', index=False)

In [37]:
xgb_pipeline.fit(X, Y)

x_test = test_df[baseline_features]
y_test_predicted = xgb_pipeline.predict(x_test)

y_test_predicted = np.rint(y_test_predicted).astype(int)

submission_df = pd.DataFrame({
    'Id': test_df['Id'],
    'SalePrice': y_test_predicted,
})

submission_df.to_csv('./data/submission_xgb.csv', index=False)

In [38]:
stacking_regressor_pipeline.fit(X, Y)

x_test = test_df[baseline_features]
y_test_predicted = stacking_regressor_pipeline.predict(x_test)

y_test_predicted = np.rint(y_test_predicted).astype(int)

submission_df = pd.DataFrame({
    'Id': test_df['Id'],
    'SalePrice': y_test_predicted,
})

submission_df.to_csv('./data/submission_stacking.csv', index=False)